# Carregando a bases de dados

In [3]:
import pandas as pd
dataset1= pd.read_csv("Divvy_Trips_2017_Q3.csv") 
dataset2= pd.read_csv("Divvy_Trips_2017_Q4.csv") 
datamaps= pd.read_csv("Divvy_Stations_2017_Q3Q4.csv")

# Merge entre as  duas  base de dados

In [4]:
dataset = pd.merge(dataset1,dataset2,how='outer')

In [5]:
# extrair from_station_id e from_station_name da  base   dataset
from_station = {'from_station_id':dataset.from_station_id,
                'from_station_name': dataset.from_station_name,
                'to_station_name': dataset.to_station_name
               }

In [6]:
partida = pd.DataFrame(data=from_station) # montar DataFrame

In [7]:
mapa = {'id':datamaps.id,
        'latitude':datamaps.latitude,
        'longitude':datamaps.longitude}

In [8]:
partida_mapa = pd.DataFrame(data=mapa)

# concatena dois DataFrame  com ID  correspondente  from_station_id

In [9]:
result = pd.concat([partida_mapa,partida], axis=1,
                   join_axes=[partida.from_station_id])

In [10]:
result=result.drop('id', axis=1) 

In [11]:
result.index.names = ['index']

# pesquisa por estacao 

In [12]:
                                                      # posicao na lista
Endereco_estacoes= ['Streeter Dr & Grand Ave',             #  0
                    'Mies van der Rohe Way & Chicago Ave', #  1
                    'State St & Pearson St',               #  2
                    'Wilton Ave & Belmont Ave',            #  3
                    'Shedd Aquarium',                      #  4
                    'Southport Ave & Irving Park Rd',      #  5
                    'Clinton St & Tilden St',              #  6
                    'Milwaukee Ave & Rockwell St',         #  7
                    'Racine Ave & 18th St'                 #  8
                    ]

posicao_lista=0

In [13]:
nome_estacao_partida=Endereco_estacoes[posicao_lista]

In [14]:
# limitar o numero de instancia para que seja mais rapido o processamento
limitar_intancias=1000

In [15]:
instancias=result[result['from_station_name'] ==
                  nome_estacao_partida ].head(limitar_intancias)

In [16]:
# tratamento dos numero de instancias
if len(instancias) != limitar_intancias:
    instancias=limitar_intancias
else:
    instancias=len(instancias)  

# filtrar por estação partida e destino

In [17]:
latitude_destino=result.latitude[result[
    'from_station_name'] ==nome_estacao_partida].head(instancias)
longitude_destino=result.longitude[result[
    'from_station_name'] ==nome_estacao_partida].head(instancias)

In [18]:
destino= pd.DataFrame(data=latitude_destino)
destino1 = pd.DataFrame(data=longitude_destino)

# posicionamento no map a estação de destino

In [19]:
latitude_destino = pd.DataFrame.from_records(destino, 
                                             columns=['latitude'])
longitude_destino = pd.DataFrame.from_records(destino1,
                                              columns=['longitude'])

In [20]:
latitude_origem=datamaps.latitude[datamaps['name'] == 
                                  nome_estacao_partida]
longitude_origem=datamaps.longitude[datamaps['name'] == 
                                    nome_estacao_partida]

# transformar em lista 

In [21]:
px2=list(latitude_origem)
py2=list(longitude_origem)
px1=list(latitude_destino.latitude[0:len(latitude_destino)])
py1=list(longitude_destino.longitude[0:len(longitude_destino)])

# Gerar o mapas de trajeto

In [ ]:
%matplotlib inline 
import smopy
import matplotlib.pyplot as pp
    
map_chi = smopy.Map((41.86, -87.645, 41.93, -87.605),z=11);
ax = map_chi.show_mpl(figsize=(10, 10));
 
for t in range(len(longitude_destino)):  
    x1,y1 = map_chi.to_pixels(px1[t],py1[t]);
    x2,y2 = map_chi.to_pixels(px2[0],py2[0]);   

    ax.plot([x1, x2], [y1, y2], 'r');
    ax.plot(x1, y1, 'go');
    ax.plot(x2, y2, 'r*');    

pp.show()


# grava a imagem em PDF

In [23]:
import matplotlib.pyplot as plt
plt.savefig('trajeto.pdf') # gravar magem   em PDF


<Figure size 432x288 with 0 Axes>

# Verificar as estações de partida  para  uma estação de  chegada

In [24]:
estacoes_chegada=result[(result['from_station_name'] ==nome_estacao_partida)]
# checar as  viagem

In [25]:
estacoes_chegada = estacoes_chegada.groupby(['latitude',
                                       'longitude',
                                       'from_station_id',
                                       'from_station_name',
                                       'to_station_name']).size().reset_index()

In [26]:
estacoes_chegada.columns = ['latitude',
                            'longitude',
                            'from_station_id',
                            'from_station_name',
                            'to_station_name', 
                            'freq']

In [27]:
estacoes_chegada.drop('from_station_id', axis=1)

,latitude,longitude,from_station_name,to_station_name,freq
0,41.874754,-87.649807,Streeter Dr & Grand Ave,McClurg Ct & Erie St,195
1,41.875009,-87.633106,Streeter Dr & Grand Ave,Theater on the Lake,25410
2,41.878287,-87.643909,Streeter Dr & Grand Ave,McClurg Ct & Erie St,5158
3,41.883668,-87.648670,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave,8637
4,41.893992,-87.629318,Streeter Dr & Grand Ave,McClurg Ct & Erie St,6274
5,41.897448,-87.628722,Streeter Dr & Grand Ave,McClurg Ct & Erie St,3551
6,41.915520,-87.687022,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave,147
7,41.968354,-87.711831,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave,352
8,41.974911,-87.692503,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave,271
9,41.975633,-87.701419,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave,129
